In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import time
from tqdm import tqdm

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

driver_pathname = 'C:/Users/sujung/Downloads/chromedriver_win32/chromedriver.exe'
# driver_pathname = 'C:/Users/yonsai/Downloads/chromedriver_win32/chromedriver.exe'


### 멜론  장르(OST, 국내영화, 국내드라마, 국내뮤지컬)

In [5]:
driver = webdriver.Chrome(driver_pathname)

### 1. songId 만 가져와 파일로 저장하기

In [4]:
gn_code = [2, 4, 7]
songIds = []

base_url1 = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN1500&dtlGnrCode=GN150'
base_url2 = '#params%5BgnrCode%5D=GN1500&params%5BdtlGnrCode%5D=GN150'
base_url3 = '&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex='

# base_url = 'https://www.melon.com/genre/song_list.htm?params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&params%5BdtlGnrCode%5D=params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&gnrCode=GN0'
# genre_url = '#params%5BgnrCode%5D=GN0'
# index_url = '&startIndex='
for i in gn_code:

    for k in range(1, 452, 50):

        page_url = f'{base_url1}{i}{base_url2}{i}{base_url3}{k}'
        driver.get(page_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        trs = soup.select('tbody > tr')
        for tr in trs:
            try:
                if tr.select_one('.bullet_icons.age_19'):
                    continue
                songId = tr.select_one('.btn.button_icons.type03.song_info')['href'].split("'")[-2]
                songIds.append(songId)
            except:
                print(page_url)



In [ ]:
songid_df = pd.DataFrame(songIds, columns=['songId'])
songid_df.to_csv('data/songids.csv', index=False)

In [37]:
songid_df.shape

(1495, 1)

In [38]:
songid_df.nunique()

songId    1495
dtype: int64

##### 2. 파일을 읽어서 songId의 내용들 가져오기

In [324]:
df = pd.read_csv('data/songids.csv')

In [325]:
df.head()

,songId
0,418168
1,35609035
2,1046278
3,35609034
4,35728845


In [326]:
songIds = df.songId.values

##### 1. 방법1

In [ ]:
driver = webdriver.Chrome(driver_pathname)

infos = []
for songId in tqdm(songIds):
    song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
    try:
        # print(song_url)

        driver.get(song_url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        info = soup.select_one('#conts')

        # songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.artist_name').get_text()
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        album = info.select_one('.list > dd').get_text()
        ul = info.select('.list_person.clfix > li')
        lyricist = ul[0].select_one('.artist_name').get_text()
        composer = ul[1].select_one('.artist_name').get_text()
        date = info.select_one('dl > dd:nth-child(4)').get_text().replace('.','')
        img = info.select_one('.image_typeAll > img')['src']
        comment = info.select_one('#revCnt').get_text().replace('개', '')
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        # lyric = str(info.select_one('#d_video_summary')).split('\t\t\t\t\t\t\t')[-1].replace('\t', '').replace('<br/>', '\n').split('</div>')[0].strip()
        lyric = str(info.select_one('#d_video_summary')).split('\n')[-2].replace('\t', '').replace('<br/>', '\n').strip().replace(',', '')

        print(str(info.select_one('#d_video_summary')))
        try:
            buttons = info.select('.d_djcol_list > ul > li > .thumb > a ')
            plylstSeq = ' '.join([button['href'].split(',\'')[-1][:-3] for button in buttons])

        except:
            plylstSeq = np.nan

        # print(title, artist, genre, album, lyricist, composer, date, comment, like)

        infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                    'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                    'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})

    except:
        continue

driver.close()

##### 2. 방법2

In [2]:
def mynan(data):
    if data != '-':
        return data
    else:
        return np.nan

In [21]:
# songIds = '3718181 3414749 30573538 36480154'.split()

In [30]:
driver = webdriver.Chrome(driver_pathname)

infos = []
for songId in tqdm(songIds):
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
        driver.get(song_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        info = soup.select_one('#conts')

        if info.select_one('#d_video_summary') == None: continue
        # lyric = str(info.select_one('#d_video_summary')).split('\n')[-2].replace('\t', '').replace('<br/>', '\n').strip()
        # lyric = str(info.select_one('#d_video_summary')).split('\n')[-2].replace('\t', '').replace('<br/>', '\n').replace('\n\n', '\n').strip()
        lyric = str(info.select_one('#d_video_summary')).split('\t\t\t\t\t\t\t')[-1].replace('\t', '').replace('<br/>', '\n').split('</div>')[0].strip().replace('\n\n', '\n')
        # lyric = str(info.select_one('#d_video_summary')).split('\n')[-2].replace('\t', '').replace('<br/>', '\n').replace('\n\n', '\n').strip()

        img = info.select_one('.image_typeAll > img')['src']
        # songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.section_info > div > div.entry > div.info > div.artist').get_text().strip()
        album = info.select_one('.list > dd').get_text()
        date = mynan(info.select_one('dl > dd:nth-child(4)').get_text().replace('.', ''))
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        comment = info.select_one('#revCnt').get_text().replace('개', '').replace(',', '')


        # 작사 / 작곡
        lis = info.select('.section_prdcr > ul > li')
        # print(len(lis))
        lyricist, composer = '', '' 
        for li in lis:
            tmp = li.get_text(strip=True)
            tmp1 = tmp.find('작사')
            tmp2 = tmp.find('작곡')
            if tmp1 > 0 and not lyricist: lyricist = tmp[:tmp1]
            if tmp2 > 0 and not composer: composer = tmp[:tmp2]
        # print(lyricist)
        # print(composer)
        lyricist = mynan(lyricist)
        composer = mynan(composer)

        buttons = info.select('.d_djcol_list > ul > li > .thumb > a ')
        plylstSeq = mynan(' '.join([button['href'].split(',\'')[-1][:-3] for button in buttons]))
        infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                    'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                    'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})
    except:
        continue

driver.close()

 50%|█████     | 2/4 [00:08<00:08,  4.17s/it]

503547414 411111859


100%|██████████| 4/4 [00:13<00:00,  3.35s/it]

In [32]:
df = pd.DataFrame(infos)
df.shape, df.songId.nunique()

((2, 13), 2)

In [29]:
df.isna().sum()

songId       0
title        0
artist       0
genre        0
album        0
lyricist     0
composer     0
date         0
img          0
comment      0
like         0
lyric        0
plylstSeq    0
dtype: int64

In [333]:
# 저장하기
df.to_csv('data/melon_genre_ost.csv', index=False)
# df.to_csv('data/melon_genre_ost_한글.csv', encoding='euc-kr', index=False)

- 데이터 확인

In [34]:
df_ost = pd.read_csv('data/melon_genre_ost.csv')
df_ost.shape, df_ost.songId.nunique()

((1487, 13), 1487)

In [35]:
df_ost.lyric[0]

'햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고 \n또 갔죠 \n바람에 흔들리는 머릿결 \n내게 불어오는 그대 향기 \n예쁜 두눈도 웃음 소리도 \n모두가 내 것이었죠\n이런 사랑 이런 행복 쉽다 \n했었죠 이런 웃음 \n이런 축복 내게 쉽게 올리 없죠 \n눈물 조차 울음조차 닦지 \n못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘\n앞에서 \n그대 나를 떠나 간다해도 \n난 그댈 보낸적 없죠 \n여전히 그댄 나를 살게 하는 \n이율테니 \n이런 사랑 이런 행복 쉽다 했었죠 \n이런 웃음 이런 축복 내게 \n쉽게 올리 없죠 \n눈물 조차 울음 조차 닦지 못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘앞에서 \n그대 나를 떠나 간다해도 난 \n그댈 보낸적 없죠 \n기다림으로 다시 시작일테니 \n얼마나 사랑했는지 얼마나 \n또 울었는지 \n그대여 한순간 조차 잊지 말아요 \n거기 떠나간 그곳에서 날\n기억하며 기다려요\n하루씩 그대에게 다가가는 \n나 일테니'

In [36]:
# # df_ost.date = df_ost.date.astype(str)
# df_ost.like = df_ost.like.astype(int)
# df_ost.comment = df_ost.comment.astype(int)
df_ost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1487 entries, 0 to 1486
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   songId     1487 non-null   int64 
 1   title      1487 non-null   object
 2   artist     1487 non-null   object
 3   genre      1487 non-null   object
 4   album      1487 non-null   object
 5   lyricist   945 non-null    object
 6   composer   1004 non-null   object
 7   date       1487 non-null   int64 
 8   img        1487 non-null   object
 9   comment    1487 non-null   int64 
 10  like       1487 non-null   int64 
 11  lyric      1487 non-null   object
 12  plylstSeq  1339 non-null   object
dtypes: int64(4), object(9)
memory usage: 151.1+ KB


In [37]:
df_ost.isna().sum()

songId         0
title          0
artist         0
genre          0
album          0
lyricist     542
composer     483
date           0
img            0
comment        0
like           0
lyric          0
plylstSeq    148
dtype: int64

In [70]:

df_ost[['songId', 'title', 'artist', 'genre', 'album', 'lyricist', 'composer', 'date', 'comment', 'like']][50:61]

,songId,title,artist,genre,album,lyricist,composer,date,comment,like
51,36396522,남다른 응원가,신인류,록/메탈 국내영화,드림 OST,신온유,신온유,20230426,12,964
52,30239362,사랑한다 안한다,홍진영,성인가요/트로트 국내영화,조작된 도시 Special Song,안영민,안영민,20170209,38,10761
53,35690116,너에게로 가는 길 (영화 '동감' X 엔플라잉 (N.Flying)),엔플라잉 (N.Flying),록/메탈 국내영화,너에게로 가는 길 (영화 '동감' X 엔플라잉 (N.Flying)),김주희,방용석,20221001,24,2261
54,35561629,"아무 일도, 아무것도",송이한,발라드 국내영화,"아무 일도, 아무것도 (뷰티풀 모먼트 OST 스페셜 트랙)",이기환,이기환,20220828,2,1100
55,4412359,언젠가 누군가,이기찬,발라드 국내영화,남자가 사랑할 때 OST,김태윤,황상준,20140122,19,8627
56,418200,희재 (Edit Version),성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201,9,3578
57,35879557,내 마음 왜 이럴까,김고은,국내영화,영웅,한아름,오상준,20221201,20,1534
58,7842239,True Romance (엔딩 테마),Citizens!,국내영화,뷰티 인사이드 OST,NaN,NaN,20150904,38,26435
59,621915,날 그만 잊어요 (Guitar Ver.),거미,R&B/Soul 국내영화,내 머리 속의 지우개 OST,이영현,박경진,20041026,7,5229
60,4054936,행복을 주는 사람,한석규,발라드 국내영화,파파로티 OST,이주호,이주호,20130312,13,10553


In [69]:
df_ost[['songId', 'title', 'plylstSeq', 'lyric']].tail(5)

,songId,title,plylstSeq,lyric
1481,3658218,과거보다 행복한 과거,457897578,기억은 숨어 있어요 \n옛날 사진 찾아서 \n흩어진 조각을 맞춰봐요 \n절대 무리는...
1482,3658219,Aftershocks,457897578 501356800,날 잘도 처분하셨군 난 다시 무덤 안\n잘 모를 땐\n전기로 싹 다 태우시나봐\n날...
1483,3658227,명확한 생각을 찾아요 / 나 떨어져요 (리프라이즈),457897578,지금은 과정일 뿐이죠 \n예전에 몰랐던 맑은 느낌 \n강한 치료도 효과는 \n오래 ...
1485,4185938,An Ordinary Man (평범한 남자),457897578 417635465,또 다른 시간 속\n여기가 아닌 곳\n거기선 나만의 꿈 꿀 텐데\n평범한 마을의\n...
1486,5385236,놀라운 평범함,457897578 445034502,뭐가 이토록 내 맘 움직일까\n말론 설명할 수 없는 느낌\n차가운 바람도\n오늘은 ...


In [48]:
df_ost[df_ost.lyric == 'None'].shape

(157, 13)

In [49]:
# 가사 없는 것 지우기
df_ost.drop(df_ost.index[df_ost[df_ost.lyric == 'None'].index], inplace=True)

In [50]:
# 가수명이 이상한 데이터 : 가사 없는 것 지우니까 없어졌다.
df_ost[df_ost.artist.str.contains('\n\n\t\t')].shape

(0, 13)

In [52]:
df_ost.isna().sum()

songId         0
title          0
artist         0
genre          0
album          0
lyricist     396
composer     394
date           0
img            0
comment        0
like           0
lyric          0
plylstSeq    112
dtype: int64

In [54]:
# 최종 화일
df_ost.to_csv('data/melon_genre_ost_230527.csv', index=False)

In [55]:
df_ost.shape

(1330, 13)

In [53]:
df_ost[df_ost.lyric.isna()]['songId']

Series([], Name: songId, dtype: int64)